# Load data

<https://www.kaggle.com/c/bike-sharing-demand>

In [ ]:
import sage
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
# Load data
df = sage.datasets.bike()
feature_names = df.columns.tolist()[:-3]

In [ ]:
# Split data, with total count serving as regression target
train, test = train_test_split(
    df.values, test_size=int(0.1 * len(df.values)), random_state=123)
train, val = train_test_split(
    train, test_size=int(0.1 * len(df.values)), random_state=123)
Y_train = train[:, -1].copy()
Y_val = val[:, -1].copy()
Y_test = test[:, -1].copy()
train = train[:, :-3].copy()
val = val[:, :-3].copy()
test = test[:, :-3].copy()

# Train model

In [ ]:
import xgboost as xgb

In [ ]:
# Set up data
dtrain = xgb.DMatrix(train, label=Y_train)
dval = xgb.DMatrix(val, label=Y_val)

# Parameters
param = {
    'max_depth' : 10,
    'objective': 'reg:squarederror',
    'nthread': 4
}
evallist = [(dtrain, 'train'), (dval, 'val')]
num_round = 50

# Train
model = xgb.train(param, dtrain, num_round, evallist, verbose_eval=False)

In [ ]:
# Calculate performance
mean = np.mean(Y_train)
base_mse = np.mean((mean - Y_test) ** 2)
mse = np.mean((model.predict(xgb.DMatrix(test)) - Y_test) ** 2)

print('Base rate MSE = {:.2f}'.format(base_mse))
print('Model MSE = {:.2f}'.format(mse))

# Setup

In [ ]:
# Set up imputer
imputer = sage.MarginalImputer(model, test[:512])

# Set up estimators
permutation_estimator = sage.PermutationEstimator(imputer, 'mse', random_state=0)
parallel_permutation_estimator = sage.PermutationEstimator(imputer, 'mse', n_jobs=-1, random_state=0)
iterated_estimator = sage.IteratedEstimator(imputer, 'mse', random_state=0)
kernel_estimator = sage.KernelEstimator(imputer, 'mse', random_state=0)

# SAGE

In [ ]:
explanation1 = permutation_estimator(test, Y_test, thresh=0.02)
explanation2 = parallel_permutation_estimator(test, Y_test, thresh=0.02)
explanation3 = iterated_estimator(test, Y_test, thresh=0.02)
explanation4 = kernel_estimator(test, Y_test, thresh=0.02)

In [ ]:
explanations = [explanation1, explanation2, explanation3, explanation4]
names = ['Permutation Estimator', 'Parallel Permutation Estimator', 'Iterated Estimator', 'Kernel Estimator']

for i in range(len(explanations)):
    for j in range(i + 1, len(explanations)):
        plt.figure()
        
        plt.scatter(explanations[i].values, explanations[j].values)
        plt.plot([0, 18000], [0, 18000], linestyle=':', color='black')
        plt.xlabel(names[i])
        plt.ylabel(names[j])
        plt.tight_layout()
        plt.show()

# Shapley Effects

In [ ]:
explanation1 = permutation_estimator(test, thresh=0.02)
explanation2 = parallel_permutation_estimator(test, thresh=0.02)
explanation3 = iterated_estimator(test, thresh=0.02)
explanation4 = kernel_estimator(test, thresh=0.02)

In [ ]:
explanations = [explanation1, explanation2, explanation3, explanation4]
names = ['Permutation Estimator', 'Parallel Permutation Estimator', 'Iterated Estimator', 'Kernel Estimator']

for i in range(len(explanations)):
    for j in range(i + 1, len(explanations)):
        plt.figure()
        
        plt.scatter(explanations[i].values, explanations[j].values)
        plt.plot([0, 18000], [0, 18000], linestyle=':', color='black')
        plt.xlabel(names[i])
        plt.ylabel(names[j])
        plt.tight_layout()
        plt.show()